In [ ]:
!pip install datasets transformers peft

In [ ]:
#imports

import pandas as pd
import numpy as np
import json
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
import peft

In [33]:
my_path = '/content/drive/MyDrive/Semester1/Core Topics AI/'

annomi = pd.read_csv(my_path + "AnnoMI-full.csv")
annomi = annomi.drop(['video_title', 'video_url', 'topic', 'timestamp', 'annotator_id', 'therapist_input_exists', 'therapist_input_subtype', 'reflection_exists', 'reflection_subtype', 'question_exists', 'question_subtype', 'main_therapist_behaviour', 'client_talk_type'], axis=1)
annomi = annomi.query("mi_quality == 'high'")
annomi = annomi.drop_duplicates(subset=['transcript_id', 'utterance_id'])
annomi = annomi.drop(['mi_quality', 'utterance_id'], axis=1)
print(annomi.head())
print(annomi.shape)

   transcript_id interlocutor  \
0              0    therapist   
1              0       client   
2              0    therapist   
3              0       client   
4              0    therapist   

                                      utterance_text  
0  Thanks for filling it out. We give this form t...  
1                                              Sure.  
2  So, let's see. It looks that you put-- You dri...  
3                                            Mm-hmm.  
4  -and you usually have three to four drinks whe...  
(8839, 3)


In [ ]:
file_path = '/content/drive/MyDrive/Semester1/Core Topics AI/annomi.json'
with open(file_path, 'w') as jsonfile:
    json.dump(pre_annomi, jsonfile)

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")